In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_linear(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_relu(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('relu'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_sigmoid(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('sigmoid'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_tanh(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('tanh'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [23]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [24]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_linear(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 427us/step - 

1052/1052 [==============================] - 0s 131us/step - loss: 17.5121 - val_loss: 22.4203
Epoch 66/1000
1052/1052 [==============================] - 0s 94us/step - loss: 17.9007 - val_loss: 21.3062
Epoch 67/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.5308 - val_loss: 21.7375
Epoch 68/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.3030 - val_loss: 21.9538
Epoch 69/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.6103 - val_loss: 20.5202
Epoch 70/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.1921 - val_loss: 20.9762
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.8604 - val_loss: 20.4699
Epoch 72/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.0457 - val_loss: 21.0324
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.8498 - val_loss: 21.2650
Epoch 74/1000
1052/1052 [================

1052/1052 [==============================] - 0s 91us/step - loss: 13.9502 - val_loss: 18.3412
Epoch 141/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.1625 - val_loss: 16.4724
Epoch 142/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.7730 - val_loss: 16.4262
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.5196 - val_loss: 17.4919
Epoch 144/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.5203 - val_loss: 21.6959
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.9350 - val_loss: 18.3647
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.4172 - val_loss: 16.8601
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.6426 - val_loss: 15.8427
Epoch 148/1000
1052/1052 [==============================] - 0s 93us/step - loss: 14.0052 - val_loss: 15.8707
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 88us/step - loss: 10.5693 - val_loss: 12.5920
Epoch 216/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.5325 - val_loss: 13.1566
Epoch 217/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.6631 - val_loss: 13.0251
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.6575 - val_loss: 14.1229
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.0633 - val_loss: 14.5795
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.6085 - val_loss: 15.4930
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.5944 - val_loss: 13.3865
Epoch 222/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.8627 - val_loss: 13.5607
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.5655 - val_loss: 13.0489
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 100us/step - loss: 9.6724 - val_loss: 11.7055
Epoch 291/1000
1052/1052 [==============================] - 0s 96us/step - loss: 10.0266 - val_loss: 12.5823
Epoch 292/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.3040 - val_loss: 12.5948
Epoch 293/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.7724 - val_loss: 12.5049
Epoch 294/1000
1052/1052 [==============================] - 0s 99us/step - loss: 9.7017 - val_loss: 11.3037
Epoch 295/1000
1052/1052 [==============================] - 0s 101us/step - loss: 10.2168 - val_loss: 11.6813
Epoch 296/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.2059 - val_loss: 12.3863
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.9335 - val_loss: 11.7529
Epoch 298/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.3389 - val_loss: 11.9153
Epoch 299/1000
1052/1052 [=============

...

Epoch 745/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0167 - val_loss: 10.7716
Epoch 746/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.6005 - val_loss: 10.5015
Epoch 747/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.7958 - val_loss: 11.1376
Epoch 748/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9126 - val_loss: 11.4955
Epoch 749/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.2829 - val_loss: 10.8482
Epoch 750/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8674 - val_loss: 12.3031
Epoch 751/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9477 - val_loss: 10.7069
Epoch 752/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8977 - val_loss: 10.4319
Epoch 753/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.5566 - val_loss: 12.7834
Epoch 754/1000
1052/1052 [==

Epoch 821/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.6071 - val_loss: 12.0120
Epoch 822/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.6770 - val_loss: 10.6401
Epoch 823/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.1920 - val_loss: 12.2536
Epoch 824/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.0216 - val_loss: 10.2450
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.5658 - val_loss: 10.5270
Epoch 826/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.3654 - val_loss: 10.3877
Epoch 827/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.0132 - val_loss: 10.6601
Epoch 828/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9360 - val_loss: 10.2950
Epoch 829/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6826 - val_loss: 11.6526
Epoch 830/1000
1052/1052 [==

1052/1052 [==============================] - 0s 85us/step - loss: 8.9208 - val_loss: 10.5859
Epoch 897/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7681 - val_loss: 10.6677
Epoch 898/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6014 - val_loss: 10.2682
Epoch 899/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.0254 - val_loss: 10.2481
Epoch 900/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.7511 - val_loss: 10.1478
Epoch 901/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.5946 - val_loss: 10.2054
Epoch 902/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8400 - val_loss: 11.3193
Epoch 903/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.3674 - val_loss: 10.5698
Epoch 904/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.3836 - val_loss: 10.3372
Epoch 905/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 95us/step - loss: 9.3579 - val_loss: 11.3788
Epoch 973/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.5848 - val_loss: 10.1917
Epoch 974/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.7337 - val_loss: 10.0614
Epoch 975/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.7464 - val_loss: 10.3316
Epoch 976/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.2775 - val_loss: 11.1975
Epoch 977/1000
1052/1052 [==============================] - 0s 104us/step - loss: 9.0871 - val_loss: 10.2101
Epoch 978/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.9084 - val_loss: 10.1232
Epoch 979/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.3275 - val_loss: 12.0707
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7672 - val_loss: 10.0414
Epoch 981/1000
1052/1052 [================

[array([[-3.9293177 ,  0.19434448,  1.5833381 , -3.7679641 , -1.1117964 ],
        [ 0.34215468,  0.25556788, -0.3653869 ,  0.10949246, -0.13052316],
        [-0.56560266,  0.37892106, -0.2603686 ,  0.09333552, -0.38091692],
        [ 0.16801071, -0.09366929, -0.07857995,  0.08532921,  0.1286498 ],
        [-0.45890555,  0.18266131,  0.15687558, -2.2467766 , -1.2367002 ]],
       dtype=float32),
 array([-4.825783  ,  0.21218878,  1.256269  , -5.2095475 , -3.1770606 ],
       dtype=float32),
 array([[ 2.222594  , -2.414922  ,  2.08969   ,  1.4184452 ,  2.1564856 ,
          1.5767916 ,  2.3679087 ,  1.8407524 , -1.4496205 ,  2.4631839 ],
        [ 0.7141877 , -0.59554464,  1.0852578 ,  0.7927337 ,  1.2204171 ,
          1.4010733 ,  0.9252656 ,  0.89417356, -1.1906831 ,  1.0570639 ],
        [ 0.25556594, -1.1544007 ,  1.1331152 ,  1.0467371 ,  0.6917381 ,
          0.16497959,  1.2627496 ,  0.7112823 , -0.7310253 ,  0.35499611],
        [ 1.820605  , -2.0428133 ,  2.096184  ,  2.138927

In [25]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_linear_5th.h5')

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_relu(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 195us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 85us/step - loss: 18.8683 - val_loss: 26.2041
Epoch 68/1000
1052/1052 [==============================] - 0s 85us/step - loss: 19.6875 - val_loss: 24.4314
Epoch 69/1000
1052/1052 [==============================] - 0s 84us/step - loss: 19.5488 - val_loss: 24.6168
Epoch 70/1000
1052/1052 [==============================] - 0s 82us/step - loss: 19.0322 - val_loss: 24.5925
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 19.3543 - val_loss: 25.6978
Epoch 72/1000
1052/1052 [==============================] - 0s 87us/step - loss: 19.6506 - val_loss: 26.5610
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 19.5558 - val_loss: 24.3298
Epoch 74/1000
1052/1052 [==============================] - 0s 83us/step - loss: 20.4311 - val_loss: 24.9007
Epoch 75/1000
1052/1052 [==============================] - 0s 84us/step - loss: 20.4591 - val_loss: 25.7941
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 97us/step - loss: 18.0750 - val_loss: 21.8246
Epoch 143/1000
1052/1052 [==============================] - 0s 96us/step - loss: 17.8282 - val_loss: 21.9310
Epoch 144/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.4238 - val_loss: 20.6569
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.4624 - val_loss: 22.9550
Epoch 146/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.4941 - val_loss: 20.7706
Epoch 147/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.8075 - val_loss: 23.2813
Epoch 148/1000
1052/1052 [==============================] - 0s 84us/step - loss: 18.1232 - val_loss: 21.5155
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.0688 - val_loss: 22.2381
Epoch 150/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.7453 - val_loss: 22.1644
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 17.8643 - val_loss: 20.9072
Epoch 218/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.5537 - val_loss: 23.7740
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.5736 - val_loss: 23.6157
Epoch 220/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.8051 - val_loss: 21.0812
Epoch 221/1000
1052/1052 [==============================] - 0s 84us/step - loss: 18.5506 - val_loss: 22.6660
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.7244 - val_loss: 21.2072
Epoch 223/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.9507 - val_loss: 22.5116
Epoch 224/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.2441 - val_loss: 22.7270
Epoch 225/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.5728 - val_loss: 21.4795
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 85us/step - loss: 19.3074 - val_loss: 22.4010
Epoch 293/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.9261 - val_loss: 22.8435
Epoch 294/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.7111 - val_loss: 22.4608
Epoch 295/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.7923 - val_loss: 21.9785
Epoch 296/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.2019 - val_loss: 23.6589
Epoch 297/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.6124 - val_loss: 21.7041
Epoch 298/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.2035 - val_loss: 22.3420
Epoch 299/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.7410 - val_loss: 23.3871
Epoch 300/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.3123 - val_loss: 24.4056
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 18.0771 - val_loss: 21.4478
Epoch 743/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.8987 - val_loss: 21.7371
Epoch 744/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.6374 - val_loss: 22.0903
Epoch 745/1000
1052/1052 [==============================] - 0s 82us/step - loss: 18.1581 - val_loss: 22.7495
Epoch 746/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.9907 - val_loss: 22.8563
Epoch 747/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.6140 - val_loss: 21.4579
Epoch 748/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.5554 - val_loss: 21.7755
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.2651 - val_loss: 23.3673
Epoch 750/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.0238 - val_loss: 22.5335
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 93us/step - loss: 17.4098 - val_loss: 21.8249
Epoch 818/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.4053 - val_loss: 22.6970
Epoch 819/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.1403 - val_loss: 22.5694
Epoch 820/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.2480 - val_loss: 22.5333
Epoch 821/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.1878 - val_loss: 23.4281
Epoch 822/1000
1052/1052 [==============================] - 0s 81us/step - loss: 17.4312 - val_loss: 22.8795
Epoch 823/1000
1052/1052 [==============================] - 0s 85us/step - loss: 18.5049 - val_loss: 21.5454
Epoch 824/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.3960 - val_loss: 21.0278
Epoch 825/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.6346 - val_loss: 21.4676
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 17.4202 - val_loss: 21.8421
Epoch 893/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.0771 - val_loss: 20.8954
Epoch 894/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.3763 - val_loss: 23.1992
Epoch 895/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.9048 - val_loss: 21.9326
Epoch 896/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.8086 - val_loss: 22.2738
Epoch 897/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.1118 - val_loss: 23.2647
Epoch 898/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.3014 - val_loss: 21.8453
Epoch 899/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.3657 - val_loss: 22.0362
Epoch 900/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.1546 - val_loss: 21.6906
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 18.0070 - val_loss: 21.9007
Epoch 968/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.0967 - val_loss: 21.4691
Epoch 969/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.5592 - val_loss: 23.1847
Epoch 970/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.4533 - val_loss: 22.8793
Epoch 971/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.3224 - val_loss: 22.3195
Epoch 972/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.8242 - val_loss: 22.8563
Epoch 973/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.2497 - val_loss: 23.5219
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.4220 - val_loss: 23.4922
Epoch 975/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.8945 - val_loss: 24.8571
Epoch 976/1000
1052/1052 [========

[array([[-6.9873160e-01,  1.5001279e-01, -1.4628013e+00, -3.5770342e-01,
          2.0507312e-01],
        [-4.9871728e-02,  1.9290209e-01,  7.3607022e-01, -6.2675193e-02,
         -5.4032618e-01],
        [-6.2658209e-01, -8.7140542e-01,  4.6801928e-01,  2.5363505e+00,
         -2.0122204e+00],
        [-2.1121231e-01,  1.4494795e-01,  1.7510287e-01,  3.2499935e-02,
          7.8973666e-02],
        [-2.6619565e-01, -5.9680152e-01,  1.3265331e+00,  2.3448378e-02,
          8.8104000e-04]], dtype=float32),
 array([3.0221555 , 3.4456012 , 0.93795013, 2.8648953 , 3.3526647 ],
       dtype=float32),
 array([[-0.26704103, -0.24707033, -0.09471258,  0.9856984 ,  1.2312359 ,
         -0.5247978 , -0.95006806,  0.01280636, -0.468651  ,  1.068053  ],
        [-0.59096086, -0.20532243, -0.7789056 ,  1.881537  ,  1.1882198 ,
         -0.6440584 , -0.5324607 , -0.38427982, -0.06075155,  1.2582499 ],
        [-0.640467  , -0.8248334 , -0.42673445,  1.0341029 ,  0.12141625,
         -0.39100662, -0

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_relu_5th.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_sigmoid(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 224us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 16.1480 - val_loss: 19.0659
Epoch 67/1000
1052/1052 [==============================] - 0s 85us/step - loss: 15.9068 - val_loss: 18.9217
Epoch 68/1000
1052/1052 [==============================] - 0s 85us/step - loss: 15.6012 - val_loss: 18.7422
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.4531 - val_loss: 18.5162
Epoch 70/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.3021 - val_loss: 18.5573
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.1915 - val_loss: 18.1919
Epoch 72/1000
1052/1052 [==============================] - 0s 83us/step - loss: 15.0651 - val_loss: 18.3198
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.9384 - val_loss: 18.1905
Epoch 74/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.8049 - val_loss: 18.1837
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 84us/step - loss: 7.2874 - val_loss: 10.1388
Epoch 142/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.3447 - val_loss: 8.9122
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.0629 - val_loss: 9.1515
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.8834 - val_loss: 8.5647
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.9300 - val_loss: 8.8738
Epoch 146/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.7808 - val_loss: 8.4981
Epoch 147/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.7756 - val_loss: 8.4035
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.6852 - val_loss: 8.6040
Epoch 149/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.6885 - val_loss: 8.7803
Epoch 150/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 86us/step - loss: 6.0718 - val_loss: 8.1308
Epoch 218/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.6886 - val_loss: 7.4797
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4163 - val_loss: 7.6372
Epoch 220/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.7100 - val_loss: 7.4743
Epoch 221/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6131 - val_loss: 7.5329
Epoch 222/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.5485 - val_loss: 8.4684
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7330 - val_loss: 7.3354
Epoch 224/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4835 - val_loss: 7.2429
Epoch 225/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5067 - val_loss: 7.8623
Epoch 226/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 83us/step - loss: 5.2161 - val_loss: 7.0414
Epoch 294/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3477 - val_loss: 6.8512
Epoch 295/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.1746 - val_loss: 8.0568
Epoch 296/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.5069 - val_loss: 7.1471
Epoch 297/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4890 - val_loss: 7.1959
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.3510 - val_loss: 7.5051
Epoch 299/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.3061 - val_loss: 7.6202
Epoch 300/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.3646 - val_loss: 7.8938
Epoch 301/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.3755 - val_loss: 7.4250
Epoch 302/1000
1052/1052 [==========================

...

1052/1052 [==============================] - 0s 97us/step - loss: 4.1794 - val_loss: 4.8402
Epoch 750/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.1246 - val_loss: 5.2624
Epoch 751/1000
1052/1052 [==============================] - 0s 85us/step - loss: 4.2040 - val_loss: 4.6465
Epoch 752/1000
1052/1052 [==============================] - 0s 90us/step - loss: 3.9827 - val_loss: 5.5250
Epoch 753/1000
1052/1052 [==============================] - 0s 121us/step - loss: 4.0868 - val_loss: 4.6009
Epoch 754/1000
1052/1052 [==============================] - 0s 82us/step - loss: 4.3358 - val_loss: 4.8691
Epoch 755/1000
1052/1052 [==============================] - 0s 83us/step - loss: 4.2396 - val_loss: 4.8656
Epoch 756/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.3977 - val_loss: 4.6880
Epoch 757/1000
1052/1052 [==============================] - 0s 85us/step - loss: 4.0156 - val_loss: 5.2172
Epoch 758/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 88us/step - loss: 4.1446 - val_loss: 5.1212
Epoch 826/1000
1052/1052 [==============================] - 0s 84us/step - loss: 4.0356 - val_loss: 5.9217
Epoch 827/1000
1052/1052 [==============================] - 0s 83us/step - loss: 3.8250 - val_loss: 4.7476
Epoch 828/1000
1052/1052 [==============================] - 0s 80us/step - loss: 4.2660 - val_loss: 4.9897
Epoch 829/1000
1052/1052 [==============================] - 0s 83us/step - loss: 4.0416 - val_loss: 5.0879
Epoch 830/1000
1052/1052 [==============================] - 0s 85us/step - loss: 3.9572 - val_loss: 5.3680
Epoch 831/1000
1052/1052 [==============================] - 0s 87us/step - loss: 4.1266 - val_loss: 7.0692
Epoch 832/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.0285 - val_loss: 5.7040
Epoch 833/1000
1052/1052 [==============================] - 0s 86us/step - loss: 3.9382 - val_loss: 5.1633
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 84us/step - loss: 4.1683 - val_loss: 5.4954
Epoch 902/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.0843 - val_loss: 5.7889
Epoch 903/1000
1052/1052 [==============================] - 0s 86us/step - loss: 3.9305 - val_loss: 5.1204
Epoch 904/1000
1052/1052 [==============================] - 0s 84us/step - loss: 3.8271 - val_loss: 5.0444
Epoch 905/1000
1052/1052 [==============================] - 0s 85us/step - loss: 3.9745 - val_loss: 5.5551
Epoch 906/1000
1052/1052 [==============================] - 0s 84us/step - loss: 4.0793 - val_loss: 5.0271
Epoch 907/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.0020 - val_loss: 5.5026
Epoch 908/1000
1052/1052 [==============================] - 0s 84us/step - loss: 3.9614 - val_loss: 5.5146
Epoch 909/1000
1052/1052 [==============================] - 0s 84us/step - loss: 4.2321 - val_loss: 5.0888
Epoch 910/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 84us/step - loss: 3.8003 - val_loss: 5.8751
Epoch 978/1000
1052/1052 [==============================] - 0s 83us/step - loss: 3.8578 - val_loss: 5.7667
Epoch 979/1000
1052/1052 [==============================] - 0s 84us/step - loss: 3.8387 - val_loss: 5.1293
Epoch 980/1000
1052/1052 [==============================] - 0s 81us/step - loss: 3.9534 - val_loss: 5.3133
Epoch 981/1000
1052/1052 [==============================] - 0s 82us/step - loss: 3.7836 - val_loss: 5.2182
Epoch 982/1000
1052/1052 [==============================] - 0s 83us/step - loss: 3.8357 - val_loss: 5.7758
Epoch 983/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.2172 - val_loss: 5.4212
Epoch 984/1000
1052/1052 [==============================] - 0s 87us/step - loss: 3.8916 - val_loss: 5.0464
Epoch 985/1000
1052/1052 [==============================] - 0s 86us/step - loss: 3.9044 - val_loss: 5.8627
Epoch 986/1000
1052/1052 [==========================

[array([[-0.4499823 , -0.6032244 , -1.6675066 , -1.8055677 , -5.289867  ],
        [-1.313119  , -0.98834544, -0.1783103 ,  0.88390696,  0.12180699],
        [-3.9577403 , -0.61197567, -0.07918966,  0.04764467, -0.20561747],
        [ 0.345189  ,  0.26159862,  0.12791581,  0.27076417,  0.09014586],
        [ 0.6441812 , -0.53684974, -4.9618025 , -0.26548967, -0.50958484]],
       dtype=float32),
 array([-1.1258736 ,  3.0917513 , -1.1874661 ,  0.19185622, -2.706063  ],
       dtype=float32),
 array([[ 3.834655  , -1.3020366 , -0.6857818 , 13.584686  , -0.5748337 ,
          1.4964862 , 20.162653  , -0.32345614, -4.3567634 , 20.040857  ],
        [-3.2767158 ,  0.7274077 ,  1.1485547 ,  1.0802006 ,  0.87847495,
         -0.1519382 ,  0.199678  ,  1.5249951 ,  4.2322187 ,  0.03666813],
        [ 0.8269084 , 26.03941   , -2.0812976 , -0.26940414, -0.04900074,
          1.4969497 , -1.3719857 , -0.33731174,  0.7006002 ,  2.4536626 ],
        [-2.075264  , -0.56781775, -0.6655047 ,  1.729806

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sigmoid_5th.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_tanh(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 225us/step - l

1052/1052 [==============================] - 0s 84us/step - loss: 18.3520 - val_loss: 21.4510
Epoch 67/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.0496 - val_loss: 21.1246
Epoch 68/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.5202 - val_loss: 20.5554
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.3183 - val_loss: 20.4448
Epoch 70/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.9032 - val_loss: 20.6413
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.8273 - val_loss: 20.3208
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.5059 - val_loss: 20.2267
Epoch 73/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.3597 - val_loss: 19.8368
Epoch 74/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.2450 - val_loss: 19.8345
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 87us/step - loss: 13.9036 - val_loss: 17.7333
Epoch 142/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.9631 - val_loss: 17.7544
Epoch 143/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.9442 - val_loss: 17.5723
Epoch 144/1000
1052/1052 [==============================] - ETA: 0s - loss: 13.67 - 0s 86us/step - loss: 13.9449 - val_loss: 17.7304
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.8804 - val_loss: 17.7415
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.8339 - val_loss: 17.6910
Epoch 147/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8659 - val_loss: 17.7338
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8638 - val_loss: 17.4931
Epoch 149/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.8689 - val_loss: 17.6841
Epoch 150/

1052/1052 [==============================] - 0s 86us/step - loss: 13.7368 - val_loss: 17.3039
Epoch 217/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.7342 - val_loss: 17.3939
Epoch 218/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.8111 - val_loss: 17.6890
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.8526 - val_loss: 17.4816
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.7719 - val_loss: 17.4068
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.8309 - val_loss: 17.7560
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.8285 - val_loss: 17.3269
Epoch 223/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.7687 - val_loss: 17.5263
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.7253 - val_loss: 17.5634
Epoch 225/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 13.6988 - val_loss: 17.1375
Epoch 292/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.7762 - val_loss: 17.3500
Epoch 293/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.7255 - val_loss: 17.7785
Epoch 294/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.7552 - val_loss: 17.7640
Epoch 295/1000
1052/1052 [==============================] - 0s 96us/step - loss: 13.7890 - val_loss: 17.7236
Epoch 296/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.7921 - val_loss: 17.4363
Epoch 297/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.7185 - val_loss: 17.2844
Epoch 298/1000
1052/1052 [==============================] - 0s 83us/step - loss: 13.7058 - val_loss: 17.2859
Epoch 299/1000
1052/1052 [==============================] - 0s 83us/step - loss: 13.6975 - val_loss: 17.3725
Epoch 300/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 86us/step - loss: 10.1398 - val_loss: 13.5722
Epoch 742/1000
1052/1052 [==============================] - 0s 82us/step - loss: 10.0199 - val_loss: 13.9148
Epoch 743/1000
1052/1052 [==============================] - 0s 83us/step - loss: 10.1828 - val_loss: 13.3773
Epoch 744/1000
1052/1052 [==============================] - 0s 83us/step - loss: 10.0450 - val_loss: 13.6996
Epoch 745/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.1426 - val_loss: 14.2949
Epoch 746/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.1260 - val_loss: 13.5365
Epoch 747/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.2490 - val_loss: 13.5863
Epoch 748/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.9893 - val_loss: 13.7287
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.1513 - val_loss: 14.4897
Epoch 750/1000
1052/1052 [=========

1052/1052 [==============================] - 0s 86us/step - loss: 9.9371 - val_loss: 13.6194
Epoch 817/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.9505 - val_loss: 13.3622
Epoch 818/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.8648 - val_loss: 13.4030
Epoch 819/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.9569 - val_loss: 13.5588
Epoch 820/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.0459 - val_loss: 13.9252
Epoch 821/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.0530 - val_loss: 13.7744
Epoch 822/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.0575 - val_loss: 13.6740
Epoch 823/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.9708 - val_loss: 13.5342
Epoch 824/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.9771 - val_loss: 13.3504
Epoch 825/1000
1052/1052 [==============

1052/1052 [==============================] - 0s 92us/step - loss: 10.0009 - val_loss: 13.4366
Epoch 892/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.9747 - val_loss: 13.0771
Epoch 893/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.7694 - val_loss: 13.1757
Epoch 894/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.8259 - val_loss: 13.1958
Epoch 895/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.7588 - val_loss: 13.1423
Epoch 896/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.7942 - val_loss: 13.1407
Epoch 897/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.9577 - val_loss: 13.0753
Epoch 898/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.9017 - val_loss: 13.3153
Epoch 899/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.8317 - val_loss: 13.2800
Epoch 900/1000
1052/1052 [================

1052/1052 [==============================] - 0s 89us/step - loss: 9.9710 - val_loss: 13.1540
Epoch 967/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.7793 - val_loss: 13.0303
Epoch 968/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.7335 - val_loss: 13.1251
Epoch 969/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.7855 - val_loss: 13.6873
Epoch 970/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.8243 - val_loss: 13.8806
Epoch 971/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.9112 - val_loss: 13.0205
Epoch 972/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.2940 - val_loss: 13.1962
Epoch 973/1000
1052/1052 [==============================] - 0s 84us/step - loss: 10.0319 - val_loss: 13.0805
Epoch 974/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.7902 - val_loss: 13.1093
Epoch 975/1000
1052/1052 [===============

[array([[-1.811891  , -0.19100763, -1.6472718 , -5.6467724 , -0.6736696 ],
        [-0.41259205, -0.84766024, -2.2492795 , -0.15298864, -2.644314  ],
        [-0.50958574, -3.6086032 , -3.3061728 , -0.48678327, -0.98489153],
        [ 0.48166114,  0.26646465,  0.5526344 ,  0.3458153 ,  0.34682986],
        [-0.64954734,  0.3517098 ,  0.6753315 , -0.5994751 , -1.8011643 ]],
       dtype=float32),
 array([ 2.8413658 , -0.92573786, -1.5823224 , -1.6575754 ,  0.17513597],
       dtype=float32),
 array([[  0.2698168 ,  -8.119445  ,  -0.50465095,  -1.2352878 ,
           0.5043376 ,  -0.5035528 ,   0.50583005,  -3.7141747 ,
          -0.5129782 ,  -0.505595  ],
        [  0.3239123 ,  -0.1987677 ,  -0.10578597,   1.0514723 ,
           0.10826877,  -0.10665236,   0.10535426,  -0.16914673,
          -0.12313108,  -0.10528868],
        [  2.0163848 , -13.313239  ,  -3.3376703 ,  -1.0247176 ,
           3.1271975 ,  -3.15403   ,   3.4844866 , -12.846012  ,
          -4.2315946 ,  -3.8812735 ],


In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_tanh_5th.h5')

In [32]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_linear(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 677us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 99us/step - loss: 7.3265 - val_loss: 10.4808
Epoch 70/1000
274/274 [==============================] - 0s 91us/step - loss: 7.4876 - val_loss: 10.6102
Epoch 71/1000
274/274 [==============================] - 0s 99us/step - loss: 7.4257 - val_loss: 10.7069
Epoch 72/1000
274/274 [==============================] - 0s 96us/step - loss: 7.3925 - val_loss: 10.5013
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.6220 - val_loss: 11.2493
Epoch 74/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3983 - val_loss: 10.5404
Epoch 75/1000
274/274 [==============================] - 0s 97us/step - loss: 7.3455 - val_loss: 10.2880
Epoch 76/1000
274/274 [==============================] - 0s 93us/step - loss: 7.5615 - val_loss: 11.2013
Epoch 77/1000
274/274 [==============================] - 0s 95us/step - loss: 7.4672 - val_loss: 10.9513
Epoch 78/1000
274/274 [==============================] 

Epoch 147/1000
274/274 [==============================] - 0s 92us/step - loss: 6.7004 - val_loss: 9.6421
Epoch 148/1000
274/274 [==============================] - 0s 87us/step - loss: 6.8200 - val_loss: 9.2582
Epoch 149/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8051 - val_loss: 9.4428
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7561 - val_loss: 9.1250
Epoch 151/1000
274/274 [==============================] - 0s 90us/step - loss: 6.6069 - val_loss: 9.3001
Epoch 152/1000
274/274 [==============================] - 0s 89us/step - loss: 6.6678 - val_loss: 9.4046
Epoch 153/1000
274/274 [==============================] - 0s 116us/step - loss: 6.6114 - val_loss: 9.6004
Epoch 154/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5973 - val_loss: 9.3076
Epoch 155/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7311 - val_loss: 9.2276
Epoch 156/1000
274/274 [==============================

274/274 [==============================] - 0s 98us/step - loss: 5.9420 - val_loss: 8.1505
Epoch 225/1000
274/274 [==============================] - 0s 91us/step - loss: 5.9496 - val_loss: 8.2383
Epoch 226/1000
274/274 [==============================] - 0s 87us/step - loss: 5.9375 - val_loss: 8.2905
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1059 - val_loss: 7.9700
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0771 - val_loss: 8.0404
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0527 - val_loss: 8.1273
Epoch 230/1000
274/274 [==============================] - 0s 97us/step - loss: 5.9396 - val_loss: 8.1635
Epoch 231/1000
274/274 [==============================] - 0s 91us/step - loss: 6.1304 - val_loss: 8.2082
Epoch 232/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9388 - val_loss: 8.0098
Epoch 233/1000
274/274 [==============================] - 0s 87us/step

274/274 [==============================] - 0s 98us/step - loss: 5.8239 - val_loss: 7.9511
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0023 - val_loss: 8.0015
Epoch 304/1000
274/274 [==============================] - 0s 91us/step - loss: 6.0749 - val_loss: 7.9737
Epoch 305/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8826 - val_loss: 8.0127
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8868 - val_loss: 8.1738
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1207 - val_loss: 8.2501
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7645 - val_loss: 7.9898
Epoch 309/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7442 - val_loss: 7.9011
Epoch 310/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8123 - val_loss: 8.0376
Epoch 311/1000
274/274 [==============================] - 0s 94us/ste

...

274/274 [==============================] - 0s 87us/step - loss: 5.9548 - val_loss: 7.7016
Epoch 693/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8814 - val_loss: 8.2666
Epoch 694/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6934 - val_loss: 8.0411
Epoch 695/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6075 - val_loss: 8.4650
Epoch 696/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5145 - val_loss: 8.2352
Epoch 697/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4342 - val_loss: 8.1087
Epoch 698/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6026 - val_loss: 7.9366
Epoch 699/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9231 - val_loss: 7.9239
Epoch 700/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6774 - val_loss: 8.0615
Epoch 701/1000
274/274 [==============================] - 0s 91us/step

274/274 [==============================] - 0s 106us/step - loss: 5.4253 - val_loss: 8.0429
Epoch 771/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5524 - val_loss: 7.8949
Epoch 772/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4284 - val_loss: 8.1670
Epoch 773/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4031 - val_loss: 7.9040
Epoch 774/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3824 - val_loss: 8.5230
Epoch 775/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4391 - val_loss: 7.9832
Epoch 776/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3654 - val_loss: 7.9788
Epoch 777/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4501 - val_loss: 8.0171
Epoch 778/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4540 - val_loss: 8.1888
Epoch 779/1000
274/274 [==============================] - 0s 95

274/274 [==============================] - 0s 98us/step - loss: 5.4394 - val_loss: 7.9433
Epoch 848/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4079 - val_loss: 7.8961
Epoch 849/1000
274/274 [==============================] - 0s 94us/step - loss: 5.3427 - val_loss: 8.2727
Epoch 850/1000
274/274 [==============================] - 0s 92us/step - loss: 5.4458 - val_loss: 7.9121
Epoch 851/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4045 - val_loss: 8.1542
Epoch 852/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3052 - val_loss: 7.9957
Epoch 853/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3240 - val_loss: 8.1683
Epoch 854/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3420 - val_loss: 8.0013
Epoch 855/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4104 - val_loss: 7.8981
Epoch 856/1000
274/274 [==============================] - 0s 99us/ste

274/274 [==============================] - 0s 91us/step - loss: 5.3375 - val_loss: 8.1133
Epoch 926/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3745 - val_loss: 8.2611
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3554 - val_loss: 7.9725
Epoch 928/1000
274/274 [==============================] - 0s 89us/step - loss: 5.3389 - val_loss: 8.2663
Epoch 929/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2623 - val_loss: 8.0282
Epoch 930/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3346 - val_loss: 8.2373
Epoch 931/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2387 - val_loss: 8.2644
Epoch 932/1000
274/274 [==============================] - 0s 116us/step - loss: 5.3138 - val_loss: 8.2242
Epoch 933/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3254 - val_loss: 8.1903
Epoch 934/1000
274/274 [==============================] - 0s 97us/st

[array([[ 7.8396505e-01, -7.0646778e-02, -3.6483979e+00,  7.7198374e-01,
         -4.1958174e-01],
        [ 8.6152369e-01, -2.1799493e+00,  7.2000825e-01, -2.7529651e-01,
          2.5904194e-01],
        [ 1.2116531e+00, -5.9480852e-01, -6.4959057e-02, -2.0551500e-01,
          4.6306905e-01],
        [-2.9359292e-03,  1.9477630e+00,  2.6826727e-01, -9.4536908e-02,
          4.0557703e-01],
        [ 4.6087334e-01, -1.3469018e+00,  4.7335518e-03,  3.8572592e-01,
         -1.3947444e-01],
        [ 4.5578077e-01, -1.3517122e+00, -1.7410306e+00, -5.8969748e-01,
          2.1967623e-01],
        [-1.5459347e-01, -1.5943466e+00,  4.5279264e-01, -1.3606008e+00,
          9.2665529e-01]], dtype=float32),
 array([ 1.221022  ,  2.0641363 , -1.9928219 , -0.9194853 , -0.04051967],
       dtype=float32),
 array([[-0.45565432,  0.7982162 ,  0.62819386,  0.45449808, -0.42913687,
         -0.2766273 ,  0.23487231, -0.42321536,  0.90295565, -0.21211888],
        [ 0.13471723,  0.08703529,  0.078076

In [33]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_linear_5th.h5')

In [34]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_relu(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 779us/step - loss:

274/274 [==============================] - 0s 91us/step - loss: 7.7289 - val_loss: 7.4726
Epoch 70/1000
274/274 [==============================] - 0s 93us/step - loss: 7.5250 - val_loss: 7.4570
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 7.4565 - val_loss: 7.3459
Epoch 72/1000
274/274 [==============================] - 0s 90us/step - loss: 7.4927 - val_loss: 7.5216
Epoch 73/1000
274/274 [==============================] - 0s 91us/step - loss: 7.5104 - val_loss: 7.4210
Epoch 74/1000
274/274 [==============================] - 0s 96us/step - loss: 7.4894 - val_loss: 7.5914
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 7.5342 - val_loss: 7.3427
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6487 - val_loss: 7.7003
Epoch 77/1000
274/274 [==============================] - 0s 99us/step - loss: 7.5411 - val_loss: 7.7040
Epoch 78/1000
274/274 [==============================] - 0s 98us/step - loss

274/274 [==============================] - 0s 95us/step - loss: 7.0733 - val_loss: 7.5459
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1566 - val_loss: 7.0594
Epoch 149/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1099 - val_loss: 7.5337
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0254 - val_loss: 7.2354
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1179 - val_loss: 7.4951
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0975 - val_loss: 7.1750
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0575 - val_loss: 7.4833
Epoch 154/1000
274/274 [==============================] - 0s 97us/step - loss: 7.1912 - val_loss: 7.2507
Epoch 155/1000
274/274 [==============================] - 0s 93us/step - loss: 7.0844 - val_loss: 7.5532
Epoch 156/1000
274/274 [==============================] - 0s 95us/step

274/274 [==============================] - 0s 124us/step - loss: 6.9448 - val_loss: 7.3076
Epoch 226/1000
274/274 [==============================] - 0s 109us/step - loss: 6.9915 - val_loss: 7.2321
Epoch 227/1000
274/274 [==============================] - 0s 99us/step - loss: 6.9970 - val_loss: 7.3864
Epoch 228/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9701 - val_loss: 7.4030
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9052 - val_loss: 7.3262
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9896 - val_loss: 7.3772
Epoch 231/1000
274/274 [==============================] - 0s 87us/step - loss: 7.2363 - val_loss: 7.2101
Epoch 232/1000
274/274 [==============================] - 0s 95us/step - loss: 7.5209 - val_loss: 7.4105
Epoch 233/1000
274/274 [==============================] - 0s 84us/step - loss: 7.0792 - val_loss: 7.0580
Epoch 234/1000
274/274 [==============================] - 0s 95us/st

274/274 [==============================] - 0s 92us/step - loss: 7.0644 - val_loss: 7.7176
Epoch 304/1000
274/274 [==============================] - 0s 97us/step - loss: 6.8115 - val_loss: 7.6890
Epoch 305/1000
274/274 [==============================] - 0s 96us/step - loss: 6.6507 - val_loss: 7.5323
Epoch 306/1000
274/274 [==============================] - 0s 101us/step - loss: 6.6594 - val_loss: 7.6966
Epoch 307/1000
274/274 [==============================] - 0s 91us/step - loss: 6.8389 - val_loss: 8.4441
Epoch 308/1000
274/274 [==============================] - 0s 99us/step - loss: 6.7888 - val_loss: 7.4679
Epoch 309/1000
274/274 [==============================] - 0s 94us/step - loss: 6.8783 - val_loss: 8.1251
Epoch 310/1000
274/274 [==============================] - 0s 94us/step - loss: 6.7158 - val_loss: 7.5895
Epoch 311/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7358 - val_loss: 7.4856
Epoch 312/1000
274/274 [==============================] - 0s 98us/ste

...

Epoch 691/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8562 - val_loss: 8.4322
Epoch 692/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2706 - val_loss: 7.7024
Epoch 693/1000
274/274 [==============================] - 0s 93us/step - loss: 6.4763 - val_loss: 8.3078
Epoch 694/1000
274/274 [==============================] - 0s 100us/step - loss: 6.4020 - val_loss: 8.0774
Epoch 695/1000
274/274 [==============================] - 0s 92us/step - loss: 6.3914 - val_loss: 8.5425
Epoch 696/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7346 - val_loss: 7.7386
Epoch 697/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6504 - val_loss: 9.1078
Epoch 698/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4494 - val_loss: 7.6630
Epoch 699/1000
274/274 [==============================] - 0s 99us/step - loss: 6.5652 - val_loss: 9.9762
Epoch 700/1000
274/274 [=============================

Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9878 - val_loss: 7.8511
Epoch 770/1000
274/274 [==============================] - 0s 97us/step - loss: 6.4080 - val_loss: 7.7843
Epoch 771/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7056 - val_loss: 7.6143
Epoch 772/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3530 - val_loss: 8.1665
Epoch 773/1000
274/274 [==============================] - 0s 92us/step - loss: 6.3156 - val_loss: 7.9008
Epoch 774/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6378 - val_loss: 7.9197
Epoch 775/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3297 - val_loss: 8.1425
Epoch 776/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2775 - val_loss: 7.9325
Epoch 777/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2303 - val_loss: 7.9590
Epoch 778/1000
274/274 [=============================

274/274 [==============================] - 0s 91us/step - loss: 6.6747 - val_loss: 8.1817
Epoch 847/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3773 - val_loss: 7.8726
Epoch 848/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3844 - val_loss: 8.1758
Epoch 849/1000
274/274 [==============================] - 0s 96us/step - loss: 6.7734 - val_loss: 8.1971
Epoch 850/1000
274/274 [==============================] - 0s 93us/step - loss: 7.0182 - val_loss: 7.6814
Epoch 851/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6126 - val_loss: 8.2094
Epoch 852/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3634 - val_loss: 8.1108
Epoch 853/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2682 - val_loss: 7.8008
Epoch 854/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3845 - val_loss: 8.9435
Epoch 855/1000
274/274 [==============================] - 0s 98us/step

Epoch 924/1000
274/274 [==============================] - 0s 120us/step - loss: 6.3170 - val_loss: 7.6562
Epoch 925/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2089 - val_loss: 7.7639
Epoch 926/1000
274/274 [==============================] - 0s 109us/step - loss: 6.2354 - val_loss: 8.1380
Epoch 927/1000
274/274 [==============================] - 0s 87us/step - loss: 6.4804 - val_loss: 7.6714
Epoch 928/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3675 - val_loss: 8.2590
Epoch 929/1000
274/274 [==============================] - 0s 113us/step - loss: 6.3767 - val_loss: 7.8117
Epoch 930/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3845 - val_loss: 8.1702
Epoch 931/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2606 - val_loss: 7.5039
Epoch 932/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2243 - val_loss: 8.7636
Epoch 933/1000
274/274 [=========================

[array([[-0.53232044, -0.58687955,  0.39952472,  0.17230423,  0.5017748 ],
        [ 0.9582751 ,  0.6359535 ,  0.41869873, -1.8547462 ,  0.2501738 ],
        [ 0.05215393,  0.01839157, -0.86553407, -0.53786933, -0.45027575],
        [-1.1254938 ,  0.21484661, -1.3041146 ,  0.5993762 , -1.3218354 ],
        [-0.851446  , -0.01704559,  0.07102912, -0.37940705,  0.85400087],
        [-0.1623543 ,  1.6085427 ,  0.33604884, -1.3272104 ,  0.9310007 ],
        [-1.0627977 , -0.34666207,  1.078834  , -0.30213377,  0.62023973]],
       dtype=float32),
 array([0.39717206, 2.0962946 , 0.3316032 , 1.4984368 , 1.320142  ],
       dtype=float32),
 array([[ 0.89418113, -0.18161769,  0.24381569,  0.8688638 , -0.10515603,
          0.7298871 , -0.33471155, -0.3281004 , -0.18082145,  1.0777526 ],
        [ 1.2876474 , -0.86524373,  0.69760394, -0.67396533,  1.1511638 ,
         -0.63650674, -0.7862481 , -0.48302144,  0.18977526,  0.8333433 ],
        [ 0.81513196, -1.3317782 ,  0.11486205,  0.42142197, 

In [35]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_relu_5th.h5')

In [36]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_sigmoid(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 788us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 98us/step - loss: 26.4619 - val_loss: 26.7821
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 26.2299 - val_loss: 26.5795
Epoch 70/1000
274/274 [==============================] - 0s 100us/step - loss: 25.7169 - val_loss: 25.7317
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 25.2973 - val_loss: 25.5696
Epoch 72/1000
274/274 [==============================] - 0s 96us/step - loss: 24.9314 - val_loss: 25.4408
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 24.5959 - val_loss: 25.2499
Epoch 74/1000
274/274 [==============================] - 0s 91us/step - loss: 24.3237 - val_loss: 24.8705
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 24.0277 - val_loss: 24.6214
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 23.5463 - val_loss: 24.1670
Epoch 77/1000
274/274 [======================

274/274 [==============================] - 0s 98us/step - loss: 6.5693 - val_loss: 10.0140
Epoch 146/1000
274/274 [==============================] - 0s 87us/step - loss: 6.3583 - val_loss: 9.9782
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2767 - val_loss: 9.8670
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2260 - val_loss: 9.9175
Epoch 149/1000
274/274 [==============================] - 0s 93us/step - loss: 6.2586 - val_loss: 9.8326
Epoch 150/1000
274/274 [==============================] - 0s 101us/step - loss: 6.1918 - val_loss: 9.6663
Epoch 151/1000
274/274 [==============================] - 0s 96us/step - loss: 6.2332 - val_loss: 9.8434
Epoch 152/1000
274/274 [==============================] - 0s 87us/step - loss: 6.3367 - val_loss: 9.7482
Epoch 153/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2065 - val_loss: 9.6789
Epoch 154/1000
274/274 [==============================] - 0s 95us/s

Epoch 223/1000
274/274 [==============================] - 0s 89us/step - loss: 5.4094 - val_loss: 8.4923
Epoch 224/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4129 - val_loss: 8.3885
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4986 - val_loss: 8.2805
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3737 - val_loss: 8.2392
Epoch 227/1000
274/274 [==============================] - 0s 105us/step - loss: 5.3804 - val_loss: 8.4083
Epoch 228/1000
274/274 [==============================] - 0s 117us/step - loss: 5.4506 - val_loss: 8.1916
Epoch 229/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3403 - val_loss: 8.2263
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3814 - val_loss: 8.4450
Epoch 231/1000
274/274 [==============================] - 0s 105us/step - loss: 5.3961 - val_loss: 8.4437
Epoch 232/1000
274/274 [==========================

274/274 [==============================] - 0s 98us/step - loss: 5.2228 - val_loss: 7.9350
Epoch 301/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2060 - val_loss: 7.9207
Epoch 302/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1759 - val_loss: 7.9668
Epoch 303/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2167 - val_loss: 7.9545
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4042 - val_loss: 8.0640
Epoch 305/1000
274/274 [==============================] - 0s 103us/step - loss: 5.2837 - val_loss: 7.8062
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2419 - val_loss: 8.0737
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3104 - val_loss: 7.8240
Epoch 308/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3645 - val_loss: 8.0915
Epoch 309/1000
274/274 [==============================] - 0s 97u

...

274/274 [==============================] - 0s 87us/step - loss: 4.8001 - val_loss: 8.7989
Epoch 764/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6677 - val_loss: 8.6554
Epoch 765/1000
274/274 [==============================] - 0s 97us/step - loss: 4.6236 - val_loss: 8.7177
Epoch 766/1000
274/274 [==============================] - 0s 95us/step - loss: 4.6252 - val_loss: 8.8389
Epoch 767/1000
274/274 [==============================] - 0s 105us/step - loss: 4.6184 - val_loss: 8.8388
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6372 - val_loss: 8.7969
Epoch 769/1000
274/274 [==============================] - 0s 94us/step - loss: 4.6295 - val_loss: 8.7543
Epoch 770/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6321 - val_loss: 8.8150
Epoch 771/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6486 - val_loss: 8.6370
Epoch 772/1000
274/274 [==============================] - 0s 101u

Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.6274 - val_loss: 8.7189
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6146 - val_loss: 8.6937
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5830 - val_loss: 8.7309
Epoch 844/1000
274/274 [==============================] - 0s 96us/step - loss: 4.6242 - val_loss: 8.5722
Epoch 845/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6124 - val_loss: 8.6389
Epoch 846/1000
274/274 [==============================] - 0s 100us/step - loss: 4.6272 - val_loss: 8.7599
Epoch 847/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6863 - val_loss: 8.9505
Epoch 848/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8767 - val_loss: 8.8111
Epoch 849/1000
274/274 [==============================] - 0s 95us/step - loss: 4.6228 - val_loss: 8.8874
Epoch 850/1000
274/274 [=============================

Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6376 - val_loss: 8.8067
Epoch 920/1000
274/274 [==============================] - 0s 94us/step - loss: 4.6195 - val_loss: 8.8038
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5903 - val_loss: 8.8884
Epoch 922/1000
274/274 [==============================] - 0s 97us/step - loss: 4.6319 - val_loss: 8.7818
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5674 - val_loss: 8.6690
Epoch 924/1000
274/274 [==============================] - ETA: 0s - loss: 3.881 - 0s 95us/step - loss: 4.5851 - val_loss: 8.7248
Epoch 925/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5862 - val_loss: 8.8858
Epoch 926/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5701 - val_loss: 8.7051
Epoch 927/1000
274/274 [==============================] - 0s 96us/step - loss: 4.5804 - val_loss: 8.8492
Epoch 928/1000
274/274 [=====

274/274 [==============================] - 0s 87us/step - loss: 4.6635 - val_loss: 8.8625
Epoch 997/1000
274/274 [==============================] - 0s 91us/step - loss: 4.5266 - val_loss: 8.8804
Epoch 998/1000
274/274 [==============================] - 0s 103us/step - loss: 4.5697 - val_loss: 8.7742
Epoch 999/1000
274/274 [==============================] - 0s 113us/step - loss: 4.5273 - val_loss: 8.8754
Epoch 1000/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5307 - val_loss: 8.8573
7.866986210063352


[array([[-2.888967  ,  0.24963434, -1.8840702 , -0.7922792 , -1.142299  ],
        [-0.6400192 ,  1.0748415 , -1.1081294 ,  0.9746161 , -1.7024589 ],
        [ 0.4886785 ,  0.05848885,  3.134974  , -2.2829292 , -1.4802653 ],
        [-1.5796264 , -1.8094813 ,  2.5711396 , -3.7817187 , -0.07497106],
        [-0.58117926,  0.5476945 ,  0.651195  ,  0.04548264, -0.75807357],
        [ 1.1498816 ,  3.2590168 , -0.1013321 ,  0.27861935,  0.5381409 ],
        [ 0.60065675, -0.41926327, -3.3119555 ,  0.10611278, -0.7128088 ]],
       dtype=float32),
 array([0.9993308 , 0.17911163, 0.8320461 , 0.33863136, 1.3267918 ],
       dtype=float32),
 array([[-0.21750154, -0.5083548 ,  1.4419355 , -0.83671844, -1.8347126 ,
          0.9081314 ,  0.18256028,  2.272771  ,  0.33706826, -0.6974015 ],
        [-1.606229  , -1.0703155 ,  0.51851195, -0.5597823 ,  3.1134708 ,
         -0.7387621 , -1.1735978 ,  0.7468676 ,  1.546497  , -1.2376742 ],
        [-1.033086  , -0.7762064 , -0.3126401 , -0.69000643, 

In [37]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sigmoid_5th.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_tanh(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 863us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 109us/step - loss: 61.0349 - val_loss: 60.6783
Epoch 69/1000
274/274 [==============================] - 0s 101us/step - loss: 61.0320 - val_loss: 60.7162
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 61.0241 - val_loss: 60.7242
Epoch 71/1000
274/274 [==============================] - 0s 105us/step - loss: 60.9335 - val_loss: 60.5868
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 57.1365 - val_loss: 47.8685
Epoch 73/1000
274/274 [==============================] - 0s 94us/step - loss: 44.4368 - val_loss: 44.9113
Epoch 74/1000
274/274 [==============================] - 0s 95us/step - loss: 41.9065 - val_loss: 41.8178
Epoch 75/1000
274/274 [==============================] - 0s 92us/step - loss: 39.4241 - val_loss: 39.2365
Epoch 76/1000
274/274 [==============================] - 0s 100us/step - loss: 37.4821 - val_loss: 37.3355
Epoch 77/1000
274/274 [==================

274/274 [==============================] - 0s 102us/step - loss: 8.4618 - val_loss: 11.6161
Epoch 145/1000
274/274 [==============================] - 0s 103us/step - loss: 8.0654 - val_loss: 11.7063
Epoch 146/1000
274/274 [==============================] - 0s 100us/step - loss: 7.8873 - val_loss: 11.2858
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 7.7234 - val_loss: 11.8136
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7521 - val_loss: 11.1363
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4463 - val_loss: 11.0491
Epoch 150/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2897 - val_loss: 11.0864
Epoch 151/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2486 - val_loss: 11.2682
Epoch 152/1000
274/274 [==============================] - 0s 109us/step - loss: 7.1280 - val_loss: 11.0343
Epoch 153/1000
274/274 [==============================

274/274 [==============================] - 0s 98us/step - loss: 5.9228 - val_loss: 9.4696
Epoch 222/1000
274/274 [==============================] - 0s 96us/step - loss: 5.7732 - val_loss: 9.2578
Epoch 223/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7774 - val_loss: 9.3088
Epoch 224/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7956 - val_loss: 9.4005
Epoch 225/1000
274/274 [==============================] - 0s 88us/step - loss: 5.9206 - val_loss: 9.3636
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8125 - val_loss: 9.3011
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7634 - val_loss: 9.4693
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7399 - val_loss: 9.1327
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7520 - val_loss: 9.3741
Epoch 230/1000
274/274 [==============================] - 0s 94us/step

274/274 [==============================] - 0s 95us/step - loss: 5.7062 - val_loss: 9.6505
Epoch 300/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6900 - val_loss: 9.4716
Epoch 301/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6854 - val_loss: 9.6156
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6194 - val_loss: 9.4996
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7051 - val_loss: 9.6629
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5307 - val_loss: 9.4946
Epoch 305/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5023 - val_loss: 9.1750
Epoch 306/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4925 - val_loss: 9.3947
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4854 - val_loss: 9.3220
Epoch 308/1000
274/274 [==============================] - 0s 91us/step

...

274/274 [==============================] - 0s 95us/step - loss: 5.0516 - val_loss: 9.0057
Epoch 767/1000
274/274 [==============================] - 0s 100us/step - loss: 5.1397 - val_loss: 8.8603
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0324 - val_loss: 9.1346
Epoch 769/1000
274/274 [==============================] - 0s 96us/step - loss: 5.0222 - val_loss: 8.7252
Epoch 770/1000
274/274 [==============================] - 0s 97us/step - loss: 5.2249 - val_loss: 9.3857
Epoch 771/1000
274/274 [==============================] - 0s 105us/step - loss: 5.1210 - val_loss: 8.8124
Epoch 772/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2132 - val_loss: 9.3216
Epoch 773/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4206 - val_loss: 8.7580
Epoch 774/1000
274/274 [==============================] - 0s 99us/step - loss: 5.3226 - val_loss: 9.0675
Epoch 775/1000
274/274 [==============================] - 0s 95us/st

274/274 [==============================] - 0s 95us/step - loss: 5.0708 - val_loss: 9.0717
Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0611 - val_loss: 8.9085
Epoch 846/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0799 - val_loss: 9.0511
Epoch 847/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0318 - val_loss: 9.0779
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0460 - val_loss: 8.9652
Epoch 849/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0485 - val_loss: 8.9178
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9552 - val_loss: 8.8197
Epoch 851/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9882 - val_loss: 9.0707
Epoch 852/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0931 - val_loss: 8.8171
Epoch 853/1000
274/274 [==============================] - ETA: 0s - l

274/274 [==============================] - 0s 98us/step - loss: 4.9754 - val_loss: 8.7554
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0901 - val_loss: 8.7683
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0726 - val_loss: 9.0351
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9694 - val_loss: 8.8094
Epoch 925/1000
274/274 [==============================] - 0s 96us/step - loss: 5.1491 - val_loss: 8.8573
Epoch 926/1000
274/274 [==============================] - 0s 91us/step - loss: 4.9966 - val_loss: 8.9094
Epoch 927/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9679 - val_loss: 8.9145
Epoch 928/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1505 - val_loss: 9.1484
Epoch 929/1000
274/274 [==============================] - 0s 87us/step - loss: 5.0622 - val_loss: 8.7869
Epoch 930/1000
274/274 [==============================] - 0s 98us/ste

274/274 [==============================] - 0s 92us/step - loss: 5.0738 - val_loss: 8.6573
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9371 - val_loss: 8.9939
6.303745310185319


[array([[-0.24589933, -0.4381972 , -1.6227452 , -0.603289  , -0.52485555],
        [ 0.91198945, -1.4688308 , -0.3214242 , -0.79096806, -1.4117553 ],
        [-1.1259958 , -0.22375403,  0.8041433 ,  0.29059538, -0.18910547],
        [-0.58695287, -1.1370027 ,  0.17402044,  3.852136  ,  1.3392057 ],
        [ 0.46357173, -0.01634329,  0.13015084,  0.10417926,  0.546555  ],
        [ 2.3118691 , -0.71987253, -0.30418858, -1.1095387 ,  0.541116  ],
        [-0.5996107 ,  0.5845429 ,  0.3385425 , -1.5930628 , -1.9905351 ]],
       dtype=float32),
 array([ 0.44093534,  0.72339076, -2.094088  , -0.9322232 , -1.6822913 ],
       dtype=float32),
 array([[  1.7369509 ,   0.71561617,   0.40615484,  -1.7409168 ,
           0.01267235,  -3.9111621 ,  -6.4561143 ,   0.13799948,
         -12.427966  ,   4.881072  ],
        [  0.9748949 ,   0.06858849,   0.29639632,  -0.531547  ,
           3.8811436 ,  -0.07089885,  -0.01796756,   9.955484  ,
          -1.7255751 ,  -0.04450022],
        [  0.30254

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_tanh_5th.h5')

In [40]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_linear(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 0

143/143 [==============================] - 0s 109us/step - loss: 0.0061 - val_loss: 0.0047
Epoch 70/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0042
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0049
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0039
Epoch 73/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0059 - val_loss: 0.0055
Epoch 74/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0069 - val_loss: 0.0040
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0064 - val_loss: 0.0046
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0063 - val_loss: 0.0045
Epoch 77/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0059 - val_loss: 0.0038
Epoch 78/1000
143/143 [==============================] - 0s 106us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0050 - val_loss: 0.0043
Epoch 147/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0047 - val_loss: 0.0033
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0038
Epoch 149/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0048 - val_loss: 0.0043
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0034
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0033
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0057
Epoch 153/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0063 - val_loss: 0.0039
Epoch 154/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0036
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0034
Epoch 224/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0041 - val_loss: 0.0029
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 226/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0032
Epoch 227/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0036 - val_loss: 0.0034
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0032
Epoch 229/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0035 - val_loss: 0.0035
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0028
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0037 - val_loss: 0.0040
Epoch 232/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 133us/step - loss: 0.0032 - val_loss: 0.0033
Epoch 301/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0034 - val_loss: 0.0037
Epoch 302/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0039 - val_loss: 0.0057
Epoch 303/1000
143/143 [==============================] - ETA: 0s - loss: 0.005 - 0s 118us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 305/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0043 - val_loss: 0.0075
Epoch 306/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0044 - val_loss: 0.0035
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0038
Epoch 308/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0036 - val_loss: 0.0037
Epoch 309/1000
143/143 [=============

...

143/143 [==============================] - 0s 110us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 763/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0065
Epoch 765/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0038
Epoch 767/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0051
Epoch 769/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 770/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 102us/step - loss: 0.0013 - val_loss: 0.0046
Epoch 840/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 841/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 842/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0037
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0040
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0063
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 846/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0018 - val_loss: 0.0040
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0034
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 115us/step - loss: 0.0012 - val_loss: 0.0056
Epoch 917/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 918/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 919/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 922/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0015 - val_loss: 0.0056
Epoch 923/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0016 - val_loss: 0.0052
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 121us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 994/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0046
Epoch 995/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0010 - val_loss: 0.0056
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 997/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0013 - val_loss: 0.0044
Epoch 998/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0011 - val_loss: 0.0049
Epoch 999/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 1000/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0012 - val_loss: 0.0044
0.017513690516352654


[array([[ 0.23961212, -0.10005765,  0.4337188 , -0.7782677 , -0.6052689 ],
        [ 0.40180287,  0.8586173 , -0.72573864,  0.13209698,  0.42037785],
        [-0.48923054, -0.00780849,  0.20838998,  0.7869779 ,  0.6560765 ],
        [ 0.1757683 , -0.3669295 ,  0.67687535,  0.6000651 ,  0.7926839 ],
        [-0.5199921 , -0.8445304 , -0.39495495,  0.1897551 ,  1.1849545 ],
        [-0.18393384,  0.20032422, -0.90306413, -1.0262172 , -0.47639707],
        [ 0.20471497,  1.045668  , -0.14460598,  0.37539184, -0.4446468 ],
        [-0.1378106 , -0.25243625,  0.5125994 ,  0.34403026, -0.8605147 ],
        [-0.4853744 ,  0.16694611,  0.469519  , -0.19028147, -1.0112673 ],
        [ 0.9531317 ,  1.176241  , -0.10195202, -0.37183577,  0.38542664],
        [-0.31505388,  0.32247886, -0.05885102,  0.23003154, -0.03723168],
        [ 0.79285026, -0.08707581, -0.24562912, -0.27454564,  0.4054678 ],
        [ 0.49495333, -0.2833381 , -0.79554164, -0.5960141 , -0.49090853],
        [-0.7759226 , -0.

In [41]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_linear_5th.h5')

In [42]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_relu(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0048 - val_loss: 0.0031
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0027
Epoch 71/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0042 - val_loss: 0.0025
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0040 - val_loss: 0.0027
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0025
Epoch 75/1000
143/143 [==============================] - 0s 94us/step - loss: 0.0039 - val_loss: 0.0028
Epoch 76/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0037 - val_loss: 0.0025
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0035 - val_loss: 0.0030
Epoch 78/1000
143/143 [==============================] - 0s 98us/step -

Epoch 147/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0033 - val_loss: 0.0023
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0027
Epoch 149/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0021
Epoch 150/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0032 - val_loss: 0.0029
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0020
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0026
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0022
Epoch 154/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0032 - val_loss: 0.0021
Epoch 155/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0025
Epoch 156/1000
143/143 [======================

143/143 [==============================] - 0s 110us/step - loss: 0.0030 - val_loss: 0.0020
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0024
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0024
Epoch 227/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0027 - val_loss: 0.0021
Epoch 228/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0021
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 230/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0030 - val_loss: 0.0020
Epoch 231/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0029 - val_loss: 0.0025
Epoch 232/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0019
Epoch 233/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0025
Epoch 302/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0026 - val_loss: 0.0025
Epoch 303/1000
143/143 [==============================] - 0s 103us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0023
Epoch 305/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0026 - val_loss: 0.0024
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0024
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0024
Epoch 308/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0025 - val_loss: 0.0023
Epoch 309/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0027 - val_loss: 0.0026
Epoch 310/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 111us/step - loss: 0.0024 - val_loss: 0.0032
Epoch 764/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0032
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0032
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0034
Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 769/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 771/1000
143/143 [==============================] - ETA: 0s - loss: 0.001 - 0s 112us/step - loss: 0.0023 - val_loss: 0.0033
Epoch 772/1000
143/143 [=============

143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0030
Epoch 841/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0026 - val_loss: 0.0029
Epoch 842/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0034
Epoch 843/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 844/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 845/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0026 - val_loss: 0.0033
Epoch 846/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0030
Epoch 847/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 848/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 849/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0030
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0031
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0035
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0024 - val_loss: 0.0031
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 923/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 925/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0030
Epoch 926/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 111us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 995/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 996/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0026 - val_loss: 0.0034
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0031
Epoch 998/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0025 - val_loss: 0.0033
Epoch 999/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 1000/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0024 - val_loss: 0.0034
0.01841886341571808


[array([[-0.2553444 , -0.6048329 ,  0.13925914, -0.2901107 ,  0.97003555],
        [-0.44323468, -0.45346352,  0.14030094,  0.5024598 ,  0.0510693 ],
        [-0.38616627, -0.10870995, -0.54246926,  0.43301463, -0.65756726],
        [ 0.14545336,  0.16296998,  0.01860991,  0.22750637, -0.26175448],
        [-0.10918033,  0.11942675, -0.29830548,  0.14449613, -0.48625788],
        [ 0.15462545, -0.40102383, -0.255068  ,  0.01886888,  0.96722656],
        [ 0.09779498, -0.4672329 , -0.06711988,  0.20488538, -0.3621773 ],
        [-0.4156864 , -0.67918634, -0.46295008,  0.6745928 ,  0.41088542],
        [-0.23709333,  0.05634768, -0.37213194,  0.28506818,  0.00508556],
        [ 0.15373012, -0.04981105, -0.21973981,  0.336623  ,  0.523039  ],
        [-0.17774528,  0.00672319, -0.7691115 ,  0.6482836 , -0.1977602 ],
        [-0.35601217, -0.6474001 , -0.51383847,  0.25640738,  0.20365772],
        [-0.06519881, -0.12917477, -0.6707338 , -0.292353  ,  0.08296744],
        [-0.2157051 , -0.

In [43]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_relu_5th.h5')

In [44]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_sigmoid(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

Epoch 69/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0372 - val_loss: 0.0271
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0375 - val_loss: 0.0258
Epoch 71/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0391 - val_loss: 0.0269
Epoch 72/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0369 - val_loss: 0.0271
Epoch 73/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0373 - val_loss: 0.0266
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0281
Epoch 75/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0374 - val_loss: 0.0275
Epoch 77/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0371 - val_loss: 0.0300
Epoch 78/1000
143/143 [==============================] 

143/143 [==============================] - 0s 111us/step - loss: 0.0371 - val_loss: 0.0284
Epoch 147/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0382 - val_loss: 0.0259
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0378 - val_loss: 0.0308
Epoch 149/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0252
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0376 - val_loss: 0.0257
Epoch 151/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0371 - val_loss: 0.0289
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0374 - val_loss: 0.0263
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0391 - val_loss: 0.0249
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0410 - val_loss: 0.0321
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0276
Epoch 224/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0275
Epoch 225/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 226/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0371 - val_loss: 0.0264
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0373 - val_loss: 0.0269
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0302
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0376 - val_loss: 0.0262
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0276
Epoch 231/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 232/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 102us/step - loss: 0.0371 - val_loss: 0.0250
Epoch 301/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0378 - val_loss: 0.0267
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0373 - val_loss: 0.0283
Epoch 303/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0369 - val_loss: 0.0259
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0380 - val_loss: 0.0273
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0254
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0375 - val_loss: 0.0286
Epoch 307/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0373 - val_loss: 0.0262
Epoch 308/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 309/1000
143/143 [==============================] - 0s 1

...

143/143 [==============================] - 0s 112us/step - loss: 0.0381 - val_loss: 0.0260
Epoch 763/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 765/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0381 - val_loss: 0.0253
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0367 - val_loss: 0.0294
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0383 - val_loss: 0.0289
Epoch 768/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0365 - val_loss: 0.0250
Epoch 769/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0377 - val_loss: 0.0268
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0277
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 840/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 841/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0369 - val_loss: 0.0271
Epoch 842/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0371 - val_loss: 0.0291
Epoch 843/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0371 - val_loss: 0.0270
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0375 - val_loss: 0.0251
Epoch 845/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0377 - val_loss: 0.0285
Epoch 846/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0371 - val_loss: 0.0271
Epoch 847/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 848/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 112us/step - loss: 0.0377 - val_loss: 0.0256
Epoch 917/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0377 - val_loss: 0.0296
Epoch 918/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0375 - val_loss: 0.0271
Epoch 919/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0276
Epoch 920/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0375 - val_loss: 0.0290
Epoch 921/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0365 - val_loss: 0.0256
Epoch 922/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0377 - val_loss: 0.0252
Epoch 923/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0384 - val_loss: 0.0309
Epoch 924/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0374 - val_loss: 0.0265
Epoch 925/1000
143/143 [==============================] - ETA: 0

143/143 [==============================] - 0s 119us/step - loss: 0.0373 - val_loss: 0.0278
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0273
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0263
Epoch 996/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0368 - val_loss: 0.0278
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0291
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0374 - val_loss: 0.0259
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0277
0.054668061435222626


[array([[-0.2734856 ,  0.37082425, -0.62255883,  0.1072548 , -0.44116533],
        [-0.09796852,  0.05982301, -0.72311443, -0.08237629,  0.4259012 ],
        [-0.2411061 , -0.36536554, -0.03889912, -0.38626286, -0.23531012],
        [-0.79316694, -0.10201132, -0.5673435 ,  0.4023533 , -0.34997788],
        [-0.3339863 ,  0.17116085,  0.19883928, -0.441598  , -0.1059728 ],
        [-0.66712797,  0.02015126,  0.25129876, -0.11775672,  0.10107872],
        [-0.1000431 , -0.05776119, -0.18173249,  0.01175715, -0.34270334],
        [ 0.00133208, -0.23219587, -0.6132708 , -0.2583318 , -0.09436658],
        [-0.32274204, -0.2266752 ,  0.01882485, -0.72619116, -0.11201993],
        [-0.46944308, -0.43333152, -0.49910417, -0.3487509 ,  0.17863068],
        [-0.5165795 ,  0.46585014, -0.29075336,  0.02686631,  0.3483626 ],
        [ 0.05901462, -0.12202737, -0.26754102, -0.60565716, -0.27554104],
        [ 0.20894448, -0.2810589 , -0.5025081 , -0.26723394, -0.45516056],
        [-0.5797649 , -0.

In [45]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sigmoid_5th.h5')

In [46]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_tanh(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0059
Epoch 70/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0069
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0059
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0066
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0061
Epoch 74/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0060
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0042 - val_loss: 0.0062
Epoch 76/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0042 - val_loss: 0.0063
Epoch 77/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0042 - val_loss: 0.0067
Epoch 78/1000
143/143 [==============================] - 0s 108us/ste

143/143 [==============================] - 0s 109us/step - loss: 0.0037 - val_loss: 0.0056
Epoch 147/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0037 - val_loss: 0.0058
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0056
Epoch 149/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0038 - val_loss: 0.0061
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0054
Epoch 151/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0042 - val_loss: 0.0058
Epoch 152/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0037 - val_loss: 0.0054
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0060
Epoch 154/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0039 - val_loss: 0.0056
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0045 - val_loss: 0.0057
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0051
Epoch 225/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0039 - val_loss: 0.0053
Epoch 226/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0036 - val_loss: 0.0048
Epoch 227/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0038 - val_loss: 0.0052
Epoch 228/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0037 - val_loss: 0.0056
Epoch 229/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0037 - val_loss: 0.0048
Epoch 230/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0038 - val_loss: 0.0055
Epoch 231/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0036 - val_loss: 0.0050
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0036 - val_loss: 0.0047
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0053
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0046
Epoch 303/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0035 - val_loss: 0.0054
Epoch 304/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0036 - val_loss: 0.0049
Epoch 305/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0035 - val_loss: 0.0054
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 307/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0037 - val_loss: 0.0065
Epoch 308/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0044 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 763/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0051
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0050
Epoch 766/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0052
Epoch 767/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 768/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 770/1000
143/143 [==============================] - 0s 140us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0041 - val_loss: 0.0047
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0041 - val_loss: 0.0057
Epoch 841/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0038 - val_loss: 0.0049
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0061
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0043
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0052
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0049
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0049
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0050
Epoch 917/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0032 - val_loss: 0.0051
Epoch 918/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0051
Epoch 919/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0035 - val_loss: 0.0048
Epoch 920/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0050
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0050
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0049
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0056
Epoch 925/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 126us/step - loss: 0.0034 - val_loss: 0.0058
Epoch 994/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0033 - val_loss: 0.0046
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0057
Epoch 996/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0037 - val_loss: 0.0048
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0036 - val_loss: 0.0058
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0049
Epoch 999/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0037 - val_loss: 0.0061
Epoch 1000/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0034 - val_loss: 0.0048
0.006723756901919842


[array([[-0.5889044 ,  0.19392361, -0.4884638 , -0.09617858, -0.42845476],
        [ 0.02853404, -0.24259953, -0.32928628, -0.6170484 , -0.05783172],
        [-0.00337238,  0.17819785, -0.4647619 ,  0.24383534,  0.43667543],
        [-0.32873204, -0.08096864,  0.11507086, -0.6562619 , -0.83762246],
        [-0.6884135 , -0.02185112,  0.21333525, -0.2928    ,  0.32583094],
        [-0.7006366 ,  0.3433212 , -0.29000813, -0.02166773,  0.11607777],
        [ 0.03754661,  0.34486803, -0.5789056 , -0.6496163 ,  0.14209476],
        [-0.5123943 , -0.30852157, -0.6864574 ,  0.16015293, -0.2194032 ],
        [-0.5204131 ,  0.17659007,  0.223694  , -0.6997629 ,  0.8458895 ],
        [-0.08449412, -0.10583706, -0.62992996, -0.13268428, -0.37106642],
        [-0.69442636, -0.2689401 , -0.3025766 , -0.07355729,  0.06909021],
        [-0.209375  , -0.3966543 , -0.6346901 , -0.25619835, -1.5227519 ],
        [-0.8082905 , -0.27908117, -0.16357328, -0.3187729 , -0.22881417],
        [-0.07374743,  1.

In [47]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_tanh_5th.h5')

In [48]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_linear(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 36.161

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0271 - val_loss: 0.0102
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0249 - val_loss: 0.0076
Epoch 73/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0234 - val_loss: 0.0059
Epoch 74/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0227 - val_loss: 0.0050
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0226 - val_loss: 0.0047
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0227 - val_loss: 0.0048
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0229 - val_loss: 0.0051
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0229 - val_loss: 0.0055
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0226 - val_loss: 0.0060
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0090 - val_loss: 0.0046
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0090 - val_loss: 0.0046
Epoch 153/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0089 - val_loss: 0.0046
Epoch 154/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0089 - val_loss: 0.0046
Epoch 155/1000
32/32 [==============================] - 0s 171us/step - loss: 0.0088 - val_loss: 0.0046
Epoch 156/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0088 - val_loss: 0.0045
Epoch 157/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0087 - val_loss: 0.0045
Epoch 158/1000
32/32 [==============================] - 0s 151us/step - loss: 0.0087 - val_loss: 0.0045
Epoch 159/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0086 - val_loss: 0.0045
Epoch 160/1000
32/32 [==============================] - 0s 134us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0034
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0034
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0034
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0062 - val_loss: 0.0034
Epoch 234/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0061 - val_loss: 0.0034
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0061 - val_loss: 0.0034
Epoch 236/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0061 - val_loss: 0.0034
Epoch 237/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0061 - val_loss: 0.0034
Epoch 238/1000
32/32 [==============================] - 0s 175us/step - loss: 0.0060 - val_loss: 0.0033
Epoch 239/1000
32/32 [==============================] - 0s 136us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0027
Epoch 310/1000
32/32 [==============================] - 0s 144us/step - loss: 0.0047 - val_loss: 0.0027
Epoch 311/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0047 - val_loss: 0.0027
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 313/1000
32/32 [==============================] - 0s 141us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 315/1000
32/32 [==============================] - 0s 137us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 316/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0027
Epoch 318/1000
32/32 [==============================] - 0s 145us

...

Epoch 704/1000
32/32 [==============================] - 0s 143us/step - loss: 0.0015 - val_loss: 0.0011
Epoch 705/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0011
Epoch 706/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0011
Epoch 707/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 708/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 709/1000
32/32 [==============================] - 0s 111us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 710/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 711/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 712/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0010
Epoch 713/1000
32/32 [==============================] - 0s 156us

Epoch 781/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 8.8769e-04
Epoch 782/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0013 - val_loss: 8.8573e-04
Epoch 783/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0013 - val_loss: 8.8378e-04
Epoch 784/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 8.8181e-04
Epoch 785/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 8.7986e-04
Epoch 786/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 8.7791e-04
Epoch 787/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 8.7600e-04
Epoch 788/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0013 - val_loss: 8.7404e-04
Epoch 789/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 8.7210e-04
Epoch 790/1000
32/32 [======

Epoch 857/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.5226e-04
Epoch 858/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.5066e-04
Epoch 859/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 7.4904e-04
Epoch 860/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.4743e-04
Epoch 861/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.4585e-04
Epoch 862/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.4425e-04
Epoch 863/1000
32/32 [==============================] - 0s 175us/step - loss: 0.0011 - val_loss: 7.4265e-04
Epoch 864/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.4108e-04
Epoch 865/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 7.3950e-04
Epoch 866/1000
32/32 [======

Epoch 933/1000
32/32 [==============================] - 0s 186us/step - loss: 9.6067e-04 - val_loss: 6.4093e-04
Epoch 934/1000
32/32 [==============================] - 0s 156us/step - loss: 9.5898e-04 - val_loss: 6.3959e-04
Epoch 935/1000
32/32 [==============================] - 0s 187us/step - loss: 9.5727e-04 - val_loss: 6.3828e-04
Epoch 936/1000
32/32 [==============================] - 0s 125us/step - loss: 9.5557e-04 - val_loss: 6.3696e-04
Epoch 937/1000
32/32 [==============================] - 0s 156us/step - loss: 9.5388e-04 - val_loss: 6.3565e-04
Epoch 938/1000
32/32 [==============================] - 0s 187us/step - loss: 9.5218e-04 - val_loss: 6.3434e-04
Epoch 939/1000
32/32 [==============================] - 0s 156us/step - loss: 9.5050e-04 - val_loss: 6.3302e-04
Epoch 940/1000
32/32 [==============================] - 0s 128us/step - loss: 9.4882e-04 - val_loss: 6.3171e-04
Epoch 941/1000
32/32 [==============================] - 0s 156us/step - loss: 9.4714e-04 - val_loss: 6.3

[array([[-9.9752331e-01, -3.1555995e-01, -1.7062919e-02, -2.3388597e-01,
         -1.3761693e+00],
        [ 1.6474925e-01, -1.3092421e-01, -1.3312898e+00,  5.4147249e-01,
          6.3627553e-01],
        [ 5.8757341e-01, -1.2547808e+00, -5.5288953e-01, -1.3321853e-03,
         -8.8571090e-01]], dtype=float32),
 array([-0.76307625, -0.56359273,  0.5722596 ,  0.3922834 ,  0.48349434],
       dtype=float32),
 array([[ 0.10037835, -0.46787122,  0.27321827, -0.58838147,  0.6322562 ,
         -0.44376007,  0.27172592, -0.20047015,  0.5082623 , -0.2694613 ],
        [ 0.23398127, -0.5644493 , -0.7148327 ,  0.05694677, -0.03055131,
          0.17785366,  0.5383008 , -0.3176066 ,  0.49742344, -0.49041072],
        [-0.32708308,  0.3461977 ,  0.17793873, -0.30153468,  0.01889477,
         -0.0641147 , -0.33417505,  0.74454826, -0.05500665,  0.34119892],
        [ 0.05982511,  0.06425886,  0.35145316,  0.01230157, -0.24142796,
         -0.04392471, -0.2555049 , -0.01844395,  0.1603309 ,  0.4632

In [49]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_linear_5th.h5')

In [50]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_relu(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 39.87

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.1181 - val_loss: 0.0509
Epoch 72/1000
32/32 [==============================] - 0s 155us/step - loss: 0.1178 - val_loss: 0.0499
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.1173 - val_loss: 0.0485
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.1165 - val_loss: 0.0466
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1151 - val_loss: 0.0445
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1134 - val_loss: 0.0424
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.1115 - val_loss: 0.0406
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.1096 - val_loss: 0.0393
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1081 - val_loss: 0.0385
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0409 - val_loss: 0.0158
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0401 - val_loss: 0.0157
Epoch 153/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0394 - val_loss: 0.0156
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0386 - val_loss: 0.0156
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0380 - val_loss: 0.0155
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0377 - val_loss: 0.0155
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0374 - val_loss: 0.0156
Epoch 158/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0371 - val_loss: 0.0156
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0368 - val_loss: 0.0156
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0215 - val_loss: 0.0108
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0214 - val_loss: 0.0107
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0212 - val_loss: 0.0106
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0211 - val_loss: 0.0106
Epoch 234/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0209 - val_loss: 0.0105
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0208 - val_loss: 0.0104
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0207 - val_loss: 0.0104
Epoch 237/1000
32/32 [==============================] - 0s 217us/step - loss: 0.0205 - val_loss: 0.0103
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0204 - val_loss: 0.0102
Epoch 239/1000
32/32 [==============================] - 0s 405us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0128 - val_loss: 0.0058
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0127 - val_loss: 0.0057
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0127 - val_loss: 0.0057
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0126 - val_loss: 0.0056
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0125 - val_loss: 0.0056
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0124 - val_loss: 0.0055
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0124 - val_loss: 0.0055
Epoch 316/1000
32/32 [==============================] - 0s 130us/step - loss: 0.0123 - val_loss: 0.0054
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0122 - val_loss: 0.0054
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 768/1000
32/32 [==============================] - 0s 145us/step - loss: 1.9965e-04 - val_loss: 5.7412e-05
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9805e-04 - val_loss: 5.6618e-05
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9642e-04 - val_loss: 5.5913e-05
Epoch 771/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9483e-04 - val_loss: 5.5226e-05
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9325e-04 - val_loss: 5.4671e-05
Epoch 773/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9168e-04 - val_loss: 5.4135e-05
Epoch 774/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9013e-04 - val_loss: 5.3575e-05
Epoch 775/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8858e-04 - val_loss: 5.3000e-05
Epoch 776/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8705e-04 - val_loss: 5.2

Epoch 842/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0957e-04 - val_loss: 2.2337e-05
Epoch 843/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0869e-04 - val_loss: 2.2037e-05
Epoch 844/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0782e-04 - val_loss: 2.1736e-05
Epoch 845/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0695e-04 - val_loss: 2.1437e-05
Epoch 846/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0614e-04 - val_loss: 2.1140e-05
Epoch 847/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0525e-04 - val_loss: 2.0842e-05
Epoch 848/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0441e-04 - val_loss: 2.0548e-05
Epoch 849/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0358e-04 - val_loss: 2.0256e-05
Epoch 850/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0276e-04 - val_loss: 1.9

Epoch 916/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1240e-05 - val_loss: 7.8016e-06
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0758e-05 - val_loss: 7.6854e-06
Epoch 918/1000
32/32 [==============================] - 0s 125us/step - loss: 6.0296e-05 - val_loss: 7.5705e-06
Epoch 919/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9838e-05 - val_loss: 7.4597e-06
Epoch 920/1000
32/32 [==============================] - 0s 188us/step - loss: 5.9382e-05 - val_loss: 7.3495e-06
Epoch 921/1000
32/32 [==============================] - 0s 125us/step - loss: 5.8931e-05 - val_loss: 7.2424e-06
Epoch 922/1000
32/32 [==============================] - 0s 125us/step - loss: 5.8485e-05 - val_loss: 7.1335e-06
Epoch 923/1000
32/32 [==============================] - 0s 125us/step - loss: 5.8039e-05 - val_loss: 7.0300e-06
Epoch 924/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7599e-05 - val_loss: 6.9

Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5528e-05 - val_loss: 2.8085e-06
Epoch 991/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5280e-05 - val_loss: 2.7744e-06
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5035e-05 - val_loss: 2.7402e-06
Epoch 993/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4793e-05 - val_loss: 2.7057e-06
Epoch 994/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4550e-05 - val_loss: 2.6715e-06
Epoch 995/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4310e-05 - val_loss: 2.6369e-06
Epoch 996/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4071e-05 - val_loss: 2.6035e-06
Epoch 997/1000
32/32 [==============================] - 0s 156us/step - loss: 3.3835e-05 - val_loss: 2.5714e-06
Epoch 998/1000
32/32 [==============================] - 0s 125us/step - loss: 3.3600e-05 - val_loss: 2.5

[array([[-0.9219294 ,  0.44215757,  0.10391867, -0.37924078, -0.3759693 ],
        [ 0.47896558,  0.07363355,  0.33559403,  0.0241494 , -0.19954336],
        [-0.6338427 ,  0.0635384 , -0.40376842,  0.68505996,  0.52652735]],
       dtype=float32),
 array([ 0.3794753 ,  0.65615785, -0.9606186 , -0.42730734, -0.3283308 ],
       dtype=float32),
 array([[ 0.8603611 , -0.5180804 , -0.5690867 , -0.8689221 , -0.71206355,
          0.3129275 , -0.31382498, -0.29927102,  0.08487971, -0.28820184],
        [ 0.21627145,  0.25794956, -0.04992134, -0.43567356, -0.35210237,
          0.26018995, -0.32079503, -0.1833877 , -0.43695438, -0.91326874],
        [ 0.48736584, -0.49094352, -0.64752156, -1.0644721 , -0.47382152,
          0.1684925 , -0.55093396, -0.44319934, -0.60839975, -0.08144208],
        [-0.09109637, -1.6526837 ,  0.18868119,  0.11106791, -0.15836439,
         -0.37076148, -0.568552  , -0.51943564,  1.0294694 , -0.4451972 ],
        [ 0.3621109 , -1.0105616 ,  0.47697183,  0.2070080

In [51]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_relu_5th.h5')

In [52]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_sigmoid(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_30 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 33.36

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0054 - val_loss: 0.0076
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0067
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0060
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0031 - val_loss: 0.0054
Epoch 75/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 0.0051
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0024 - val_loss: 0.0048
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0024 - val_loss: 0.0048
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9276e-04 - val_loss: 0.0017
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 4.8733e-04 - val_loss: 0.0017
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 4.8200e-04 - val_loss: 0.0017
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7677e-04 - val_loss: 0.0017
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7164e-04 - val_loss: 0.0016
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6659e-04 - val_loss: 0.0016
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6165e-04 - val_loss: 0.0016
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 4.5680e-04 - val_loss: 0.0016
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 4.5207e-04 - val_loss: 0.0016
Epoch 158/1000
32/32 [======

32/32 [==============================] - 0s 156us/step - loss: 2.4948e-04 - val_loss: 8.1426e-04
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4739e-04 - val_loss: 8.0663e-04
Epoch 226/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4534e-04 - val_loss: 7.9908e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4330e-04 - val_loss: 7.9159e-04
Epoch 228/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4128e-04 - val_loss: 7.8416e-04
Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3928e-04 - val_loss: 7.7678e-04
Epoch 230/1000
32/32 [==============================] - 0s 176us/step - loss: 2.3729e-04 - val_loss: 7.6948e-04
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 2.3534e-04 - val_loss: 7.6224e-04
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3339e-04 - val_loss: 7.5507e-04
Epoch 2

Epoch 298/1000
32/32 [==============================] - 0s 124us/step - loss: 1.3906e-04 - val_loss: 4.0696e-04
Epoch 299/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3805e-04 - val_loss: 4.0327e-04
Epoch 300/1000
32/32 [==============================] - 0s 186us/step - loss: 1.3705e-04 - val_loss: 3.9965e-04
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3606e-04 - val_loss: 3.9604e-04
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3508e-04 - val_loss: 3.9248e-04
Epoch 303/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3411e-04 - val_loss: 3.8895e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3315e-04 - val_loss: 3.8546e-04
Epoch 305/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3220e-04 - val_loss: 3.8201e-04
Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3126e-04 - val_loss: 3.7

...

Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6502e-05 - val_loss: 4.4272e-05
Epoch 743/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6440e-05 - val_loss: 4.4132e-05
Epoch 744/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6381e-05 - val_loss: 4.3989e-05
Epoch 745/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6320e-05 - val_loss: 4.3855e-05
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6263e-05 - val_loss: 4.3716e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6202e-05 - val_loss: 4.3585e-05
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6144e-05 - val_loss: 4.3443e-05
Epoch 749/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6085e-05 - val_loss: 4.3307e-05
Epoch 750/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6027e-05 - val_loss: 4.3

Epoch 816/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2520e-05 - val_loss: 3.5119e-05
Epoch 817/1000
32/32 [==============================] - 0s 155us/step - loss: 2.2474e-05 - val_loss: 3.5011e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2424e-05 - val_loss: 3.4901e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2377e-05 - val_loss: 3.4794e-05
Epoch 820/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2330e-05 - val_loss: 3.4685e-05
Epoch 821/1000
32/32 [==============================] - 0s 155us/step - loss: 2.2281e-05 - val_loss: 3.4579e-05
Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2234e-05 - val_loss: 3.4473e-05
Epoch 823/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2187e-05 - val_loss: 3.4364e-05
Epoch 824/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2140e-05 - val_loss: 3.4

Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9321e-05 - val_loss: 2.7980e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9282e-05 - val_loss: 2.7898e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9242e-05 - val_loss: 2.7813e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9204e-05 - val_loss: 2.7729e-05
Epoch 894/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9166e-05 - val_loss: 2.7645e-05
Epoch 895/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9128e-05 - val_loss: 2.7559e-05
Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9089e-05 - val_loss: 2.7478e-05
Epoch 897/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9051e-05 - val_loss: 2.7392e-05
Epoch 898/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9013e-05 - val_loss: 2.7

Epoch 964/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6731e-05 - val_loss: 2.2382e-05
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6701e-05 - val_loss: 2.2315e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6669e-05 - val_loss: 2.2248e-05
Epoch 967/1000
32/32 [==============================] - 0s 125us/step - loss: 1.6637e-05 - val_loss: 2.2181e-05
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6606e-05 - val_loss: 2.2113e-05
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6574e-05 - val_loss: 2.2046e-05
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6543e-05 - val_loss: 2.1983e-05
Epoch 971/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6513e-05 - val_loss: 2.1914e-05
Epoch 972/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6482e-05 - val_loss: 2.1

[array([[ 1.6241488 ,  0.3950121 ,  1.0222819 , -0.38111869,  0.03728871],
        [-0.9257945 ,  0.3280026 , -0.89387   ,  0.3939142 ,  0.05936173],
        [ 1.3252648 ,  0.10865033,  1.2346674 , -0.14851665, -0.38040066]],
       dtype=float32),
 array([ 0.72371936, -0.6790009 ,  0.9120876 , -0.77194107, -0.6514658 ],
       dtype=float32),
 array([[-0.24172854, -1.073463  ,  0.9140583 ,  0.96402   ,  1.2402463 ,
         -1.3685745 ,  0.66942185, -0.89072174, -1.2900589 ,  1.2639164 ],
        [ 0.27368662, -0.45651546,  0.32009128,  0.26671144,  0.5060737 ,
         -0.48823732,  0.18656494, -0.83235097, -0.7063517 ,  1.0356672 ],
        [-1.076289  , -0.78816813,  1.5337472 ,  0.68875283,  0.3815732 ,
         -1.317498  ,  1.3186015 , -1.248862  , -1.1567262 ,  0.4609596 ],
        [ 0.48327595, -0.7269102 ,  0.4793119 ,  0.39941725,  0.6119275 ,
         -0.5928275 ,  0.2756223 , -0.6380049 , -0.18318164,  0.14729273],
        [ 0.6362725 , -0.42463973,  0.72592646,  0.6511137

In [53]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sigmoid_5th.h5')

In [54]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_tanh(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_31 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 42.51

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0029
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0018
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0013
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0015
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6814e-04 - val_loss: 0.0022
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2882e-04 - val_loss: 0.0033
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 7.6011e-04 - val_loss: 0.0046
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0056
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 0.0064
Epoch 80/1000
32/32 [==============================] - 0s 156

Epoch 148/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3791e-04 - val_loss: 0.0015
Epoch 149/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3707e-04 - val_loss: 0.0015
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3608e-04 - val_loss: 0.0015
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3506e-04 - val_loss: 0.0015
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3412e-04 - val_loss: 0.0015
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 2.3332e-04 - val_loss: 0.0015
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3270e-04 - val_loss: 0.0015
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3224e-04 - val_loss: 0.0015
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3191e-04 - val_loss: 0.0015
Epoch 157/1000
32/32 [======

Epoch 224/1000
32/32 [==============================] - 0s 125us/step - loss: 2.0466e-04 - val_loss: 0.0013
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0430e-04 - val_loss: 0.0013
Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0395e-04 - val_loss: 0.0013
Epoch 227/1000
32/32 [==============================] - 0s 155us/step - loss: 2.0360e-04 - val_loss: 0.0013
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0325e-04 - val_loss: 0.0013
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0290e-04 - val_loss: 0.0013
Epoch 230/1000
32/32 [==============================] - 0s 187us/step - loss: 2.0255e-04 - val_loss: 0.0013
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0220e-04 - val_loss: 0.0013
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0186e-04 - val_loss: 0.0013
Epoch 233/1000
32/32 [======

Epoch 300/1000
32/32 [==============================] - 0s 155us/step - loss: 1.8065e-04 - val_loss: 0.0011
Epoch 301/1000
32/32 [==============================] - 0s 125us/step - loss: 1.8037e-04 - val_loss: 0.0011
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8009e-04 - val_loss: 0.0011
Epoch 303/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7981e-04 - val_loss: 0.0011
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7953e-04 - val_loss: 0.0011
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7925e-04 - val_loss: 0.0011
Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7897e-04 - val_loss: 0.0011
Epoch 307/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7870e-04 - val_loss: 0.0011
Epoch 308/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7842e-04 - val_loss: 0.0011
Epoch 309/1000
32/32 [======

...

Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0402e-04 - val_loss: 6.0300e-04
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0391e-04 - val_loss: 6.0231e-04
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0381e-04 - val_loss: 6.0162e-04
Epoch 749/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0370e-04 - val_loss: 6.0096e-04
Epoch 750/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0360e-04 - val_loss: 6.0027e-04
Epoch 751/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0350e-04 - val_loss: 5.9959e-04
Epoch 752/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0339e-04 - val_loss: 5.9891e-04
Epoch 753/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0329e-04 - val_loss: 5.9821e-04
Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0318e-04 - val_loss: 5.9

Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6697e-05 - val_loss: 5.5547e-04
Epoch 821/1000
32/32 [==============================] - 0s 125us/step - loss: 9.6604e-05 - val_loss: 5.5487e-04
Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6513e-05 - val_loss: 5.5428e-04
Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6420e-05 - val_loss: 5.5367e-04
Epoch 824/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6327e-05 - val_loss: 5.5309e-04
Epoch 825/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6237e-05 - val_loss: 5.5249e-04
Epoch 826/1000
32/32 [==============================] - 0s 125us/step - loss: 9.6143e-05 - val_loss: 5.5191e-04
Epoch 827/1000
32/32 [==============================] - 0s 125us/step - loss: 9.6053e-05 - val_loss: 5.5131e-04
Epoch 828/1000
32/32 [==============================] - 0s 156us/step - loss: 9.5961e-05 - val_loss: 5.5

Epoch 894/1000
32/32 [==============================] - 0s 157us/step - loss: 9.0208e-05 - val_loss: 5.1393e-04
Epoch 895/1000
32/32 [==============================] - 0s 155us/step - loss: 9.0127e-05 - val_loss: 5.1340e-04
Epoch 896/1000
32/32 [==============================] - 0s 155us/step - loss: 9.0044e-05 - val_loss: 5.1287e-04
Epoch 897/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9963e-05 - val_loss: 5.1236e-04
Epoch 898/1000
32/32 [==============================] - 0s 155us/step - loss: 8.9878e-05 - val_loss: 5.1183e-04
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9796e-05 - val_loss: 5.1130e-04
Epoch 900/1000
32/32 [==============================] - 0s 125us/step - loss: 8.9713e-05 - val_loss: 5.1077e-04
Epoch 901/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9631e-05 - val_loss: 5.1025e-04
Epoch 902/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9549e-05 - val_loss: 5.0

Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4414e-05 - val_loss: 4.7733e-04
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4339e-05 - val_loss: 4.7686e-04
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4266e-05 - val_loss: 4.7642e-04
Epoch 971/1000
32/32 [==============================] - 0s 187us/step - loss: 8.4193e-05 - val_loss: 4.7595e-04
Epoch 972/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4117e-05 - val_loss: 4.7548e-04
Epoch 973/1000
32/32 [==============================] - 0s 125us/step - loss: 8.4045e-05 - val_loss: 4.7501e-04
Epoch 974/1000
32/32 [==============================] - 0s 187us/step - loss: 8.3972e-05 - val_loss: 4.7456e-04
Epoch 975/1000
32/32 [==============================] - 0s 187us/step - loss: 8.3896e-05 - val_loss: 4.7410e-04
Epoch 976/1000
32/32 [==============================] - 0s 156us/step - loss: 8.3825e-05 - val_loss: 4.7

[array([[ 0.15381368, -1.0809047 , -0.1159108 , -1.2999389 ,  0.9747637 ],
        [ 0.87885827, -0.10849021,  0.11061154,  1.6438621 , -0.6303825 ],
        [ 0.8151316 , -0.09922495,  0.05797448, -1.2769158 ,  1.0791589 ]],
       dtype=float32),
 array([ 0.36133102, -0.6233743 , -0.47601688,  0.9447642 ,  0.63551754],
       dtype=float32),
 array([[ 0.743674  , -0.8812604 , -0.43037215,  0.42600518, -0.8458779 ,
          1.2208589 ,  0.30528337,  0.819732  , -0.63974625,  0.6732348 ],
        [ 1.246652  , -0.22151151, -0.36865497,  0.32996434, -0.67205495,
          0.44576684,  0.52914613,  0.8077338 , -0.51843184,  0.48301336],
        [ 1.1811669 , -0.45939735, -0.49304518,  1.3686377 , -0.6425174 ,
          0.1828737 ,  0.48255694,  0.24234563, -0.73485154,  0.10549297],
        [ 0.6428004 , -1.0800214 , -0.7143149 ,  0.87714434, -1.4137437 ,
          0.75901186,  0.9777218 ,  0.35423937, -0.9200361 ,  1.2088847 ],
        [ 1.6090413 , -0.7208156 , -0.26685953,  1.3686955

In [55]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_tanh_5th.h5')